In [1]:
import fairbench as fb

x, y, yhat = fb.bench.tabular.compas(test_size=0.5)
print(y)
print(yhat)
print(x.keys())

[False  True False ...  True False False]
[ True  True False ...  True False False]
dict_keys(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob', 'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score', 'juv_misd_count', 'juv_other_count', 'priors_count', 'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number', 'c_offense_date', 'c_arrest_date', 'c_days_from_compas', 'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number', 'r_charge_degree', 'r_days_from_arrest', 'r_offense_date', 'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid', 'is_violent_recid', 'vr_case_number', 'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc', 'type_of_assessment', 'score_text', 'screening_date', 'v_type_of_assessment', 'v_decile_score', 'v_score_text', 'v_screening_date', 'in_custody', 'out_custody', 'start', 'end', 'event', 'two_year_recid'])


In [ ]:
x["sex"]

# Determining what to compare

In [2]:
sensitive = fb.Dimensions(fb.categories @ x["sex"])
sensitive

Male,[1 1 1 ... 1 1 1]
Female,[0 0 0 ... 0 0 0]


# Creating a report

In [3]:
report = fb.reports.pairwise(sensitive=sensitive, predictions=y, labels=yhat)
report.show(env=fb.export.ConsoleTable)

                                                                                                               
                                    multidim                                                                   
                                           acc           pr          tpr          tnr          tar          trr
min                                      0.966        0.344        0.930            1        0.344        0.501
max                                                   0.465                         1        0.465        0.630
maxerror                                 0.034                     0.070            0                          
wmean                                    0.968        0.442        0.932            1        0.442        0.526
mean                                     0.970        0.405        0.931            1        0.405        0.566
maxrel                                   0.008        0.261        0.003            0        0.261      

# Multidimensional analysis

In [6]:
sensitive = fb.Dimensions(fb.categories @ x["sex"], fb.categories @ x["race"])
sensitive = sensitive.intersectional().strict()
sensitive

Female&Other,[0 0 0 ... 0 0 0]
Female&Hispanic,[0 0 0 ... 0 0 0]
Female&African-American,[0 0 0 ... 0 0 0]
Female&Native American,[0 0 0 ... 0 0 0]
Female&Caucasian,[0 0 0 ... 0 0 0]
Male&Other,[0 0 0 ... 0 0 0]
Male&Hispanic,[0 0 0 ... 0 0 0]
Male&African-American,[1 1 1 ... 1 1 1]
Male&Native American,[0 0 0 ... 0 0 0]
Male&Caucasian,[0 0 0 ... 0 0 0]


In [7]:
multireport = fb.reports.pairwise(sensitive=sensitive, predictions=y, labels=yhat)
multireport.show(env=fb.export.ConsoleTable)

                                                                                                               
                                    multidim                                                                   
                                           acc           pr          tpr          tnr          tar          trr
min                                      0.958        0.229        0.882            0        0.229            0
max                                                       1                                      1        0.743
maxerror                                 0.042                     0.118            1                          
wmean                                    0.968        0.442        0.933        1.000        0.442        0.526
mean                                     0.976        0.438        0.937        0.900        0.438        0.538
maxrel                                   0.042        0.771        0.118            1        0.771      

# Filtering the reports

In [12]:
multireport.show(depth=10)


##### multidim #####
|This is analysis that compares several groups.
|Computations cover several cases.

 ***** min *****
 |This reduction is the minimum.
 |Computations cover several cases.
 
   ===== min acc =====
   |This reduction of a measure is the minimum of the accuracy.
   |Value: 0.958 
   
     ----- Female&Other -----
     |This is the value for group 'Female&Other'.
     |Value: 0.971 acc
     
       (0.0, 35.0)
       ▎ █            
       ▎ █            
       ▎ █     ▂     ▂
       ▎ █     █     █
       ▎ █  ▄  █  ▂  █
       ▎▬*▬▬-▬▬+▬▬x▬▬o
            (5.0, 0.0)
       
        * samples                       35 
        - ap                            9 
        + an                            26 
        x tp                            8 
        o tn                            26 
     
     ----- Female&Hispanic -----
     |This is the value for group 'Female&Hispanic'.
     |Value: 0.962 acc
     
       (0.0, 52.0)
       ▎ █            
       ▎ █         

In [11]:
multireport.std.tpr["Female&Native American"].show()


##### Female&Native American #####
|This is the value for group 'Female&Native American'.
|Value: 1.000 tpr

  (0.0, 1.0)
  ▎ █  █  █  █
  ▎ █  █  █  █
  ▎ █  █  █  █
  ▎ █  █  █  █
  ▎ █  █  █  █
  ▎▬*▬▬-▬▬+▬▬x
    (4.0, 0.0)
  
   * samples                             1 
   - positives                           1 
   + ap                                  1 
   x tp                                  1 





# Fairness model cards - common definitions with caveats

In [13]:
multireport.filter(fb.investigate.Stamps).show(depth=1, env=fb.export.Html)  # this env opens a new browser window

# Comparing reports

In [14]:
comparison = fb.Progress("Comparison")
comparison.instance("Naive", report)
comparison.instance("Intersectional", multireport)
comparison = comparison.build()
comparison.show()


##### Comparison #####
|This is tracking progress across Comparison.
|Computations cover several cases.

 ***** Naive multidim *****
 |This instance is analysis that compares several groups.
 |Computations cover several cases.
 
   ===== min =====
   |This reduction is the minimum.
   |Computations cover several cases.
   
     (0.0, 1.0)
     ▎          █      
     ▎ ▆     ▄  █      
     ▎ █     █  █      
     ▎ █     █  █     █
     ▎ █  █  █  █  █  █
     ▎▬*▬▬-▬▬+▬▬x▬▬o▬▬□
             (6.0, 0.0)
     
      * min acc                         0.966 
      - min pr                          0.344 
      + min tpr                         0.930 
      x min tnr                         1 
      o min tar                         0.344 
      □ min trr                         0.501 
   
   ===== max =====
   |This reduction is the maximum.
   |Computations cover several cases.
   
     (0.0, 1.0)
     ▎    █      
     ▎    █      
     ▎    █     ▆
     ▎ ▆  █  ▆  █
     ▎ █  █  █  █


In [16]:
comparison.std.explain.show()  # .explain changes what is grouped


##### std explain #####
|This explanation of a reduction is the standard deviation viewed for inner 
|details.
|Computations cover several cases.

 ***** acc *****
 |This measure is the accuracy.
 |Computations cover several cases.
 
   (0.0, 0.014141318572283006)
   ▎    █
   ▎    █
   ▎    █
   ▎    █
   ▎ ▄  █
   ▎▬*▬▬-
   (2.0, 0.0)
   
    * Naive multidim                    0.004 
    - Intersectional multidim           0.014 
 
 ***** pr *****
 |This measure is the positive rate.
 |Computations cover several cases.
 
   (0.0, 0.20546414021260637)
   ▎    █
   ▎    █
   ▎    █
   ▎    █
   ▎ ▆  █
   ▎▬*▬▬-
   (2.0, 0.0)
   
    * Naive multidim                    0.061 
    - Intersectional multidim           0.205 
 
 ***** tpr *****
 |This measure is the true positive rate/recall/sensitivity/hit rate.
 |Computations cover several cases.
 
   (0.0, 0.03889131825371914)
   ▎    █
   ▎    █
   ▎    █
   ▎    █
   ▎    █
   ▎▬*▬▬-
   (2.0, 0.0)
   
    * Naive multidim            

In [18]:
comparison.filter(fb.investigate.DeviationsOver(0.1)).show(env=fb.export.ConsoleTable(sideways=False))


Comparison Naive multidim
                                     maxrel      maxdiff
pr                                    0.261        0.121
tar                                   0.261        0.121
trr                                   0.205        0.129

Comparison Intersectional multidim
                                        tpr          tnr           pr          tar          trr
min                                   0.882            0                                       
maxerror                              0.118            1                                       
maxrel                                0.118            1        0.771        0.771            1
maxdiff                               0.118            1        0.771        0.771        0.743
gini                                                            0.218        0.218        0.172
std                                                0.300        0.205        0.205        0.197

